# Create a SQLite DataBase 


In [5]:
import sqlite3
import pandas as pd
import os

# Path to the SQLite database
db_path = 'Outputs/fire_data.sqlite'

# Delete the database if it already exists
if os.path.exists(db_path):
    os.remove(db_path)
    print(f"Deleted existing database at {db_path}")

# Paths to the CSV files
counties_csv_path = 'Resources/california_counties_forDB_modified.csv'
extracted_data_csv_path = 'Outputs/extracted_data_modified_cleaned_corrected.csv'  
fires_cleaned_csv_path = 'Outputs/fire_stats_year_county.csv'

# Load the CSV files
counties_df = pd.read_csv(counties_csv_path)
extracted_data_df = pd.read_csv(extracted_data_csv_path)
fires_cleaned_df = pd.read_csv(fires_cleaned_csv_path)

# Create SQLite database connection (this will create a new database file)
conn = sqlite3.connect(db_path)

# Save the dataframes to the database
counties_df.to_sql('california_counties', conn, if_exists='replace', index=False,
                   dtype={
                       'County': 'TEXT',
                       'Abbreviation': 'TEXT',
                       'county_up': 'TEXT'
                   })

extracted_data_df.to_sql('extracted_data', conn, if_exists='replace', index=False,
                         dtype={
                             'County': 'TEXT',
                             'Tot_Damage': 'REAL',
                             'Year': 'INTEGER',
                             'county_up': 'TEXT'
                         })

fires_cleaned_df.to_sql('fire_stats_year_county', conn, if_exists='replace', index=False,
                        dtype={
                            'County': 'TEXT',
                            'Fire Name': 'TEXT',
                            'Start': 'TEXT',
                            'Contained': 'TEXT',
                            'Acres': 'REAL',
                            'Deaths_FF': 'INTEGER',
                            'Deaths_Civil': 'INTEGER',
                            'Duration': 'REAL',
                            'Strux_Destr': 'INTEGER',
                            'Strux_Dmgd': 'INTEGER',
                            'Year': 'INTEGER'
                        })

# Drop the 'combined_data' table if it exists
conn.execute("DROP TABLE IF EXISTS combined_data;")

# Link the data using the 'County' field and 'county_up'
query = """
    CREATE TABLE combined_data AS
    SELECT *
    FROM fire_stats_year_county f
    JOIN california_counties c ON f.County = c.county_up
    JOIN extracted_data e ON f.County = e.county_up AND f.Year = e.Year;
"""

conn.execute(query)

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Database setup complete.")


Deleted existing database at Outputs/fire_data.sqlite
Database setup complete.


In [7]:
# Reopen the SQLite connection
conn = sqlite3.connect('Outputs/fire_data.sqlite')

# Query to find all unique fires in SISKIYOU in 2014 and include fire stats
query_siskiyou_fires = """
    SELECT DISTINCT f.County, f."Fire Name", f.Start, f.Contained, f.Acres, f.Deaths_FF, 
                    f.Deaths_Civil, f.Duration, f.Strux_Destr, f.Strux_Dmgd, f.Year
    FROM fire_stats_year_county f
    JOIN extracted_data e ON f.County = e.county_up AND f.Year = e.Year
    WHERE f.County = 'SISKIYOU' AND f.Year = 2014;
"""

# Execute the query and fetch the results
result_siskiyou_fires = pd.read_sql_query(query_siskiyou_fires, conn)

# Query to get the total damage for SISKIYOU in 2014
query_siskiyou_damage = """
    SELECT SUM(e.Tot_Damage) AS Total_Damage
    FROM extracted_data e
    WHERE e.county_up = 'SISKIYOU' AND e.Year = 2014;
"""

# Execute the query and fetch the total damage
result_siskiyou_damage = pd.read_sql_query(query_siskiyou_damage, conn)

# Closing the connection
conn.close()

# Display the results
result_siskiyou_fires, result_siskiyou_damage


(     County            Fire Name       Start   Contained     Acres  Deaths_FF  \
 0  SISKIYOU           KNF BEAVER  2014-07-30  2014-08-30   32496.0          0   
 1  SISKIYOU          LITTLE DEER  2014-07-31  2014-08-11    5503.0          0   
 2  SISKIYOU         JULY COMPLEX  2014-08-03  2014-09-25   50042.0          0   
 3  SISKIYOU  HAPPY CAMP\nCOMPLEX  2014-08-14  2014-10-31  134056.0          0   
 
    Deaths_Civil  Duration  Strux_Destr  Strux_Dmgd  Year  
 0             0      32.0            6           0  2014  
 1             0      12.0            1           0  2014  
 2             0      54.0            2           0  2014  
 3             0      79.0            6           2  2014  ,
    Total_Damage
 0           0.0)